<a href="https://colab.research.google.com/github/priyanshgupta1998/STOCK/blob/master/Web-Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
from requests import get
from csv import writer

url = 'https://www.fpi.nsdl.co.in/web/Reports/Latest.aspx'


r = get(url)
soup = BeautifulSoup(r.text, 'lxml')


# get all tables
tables = soup.find_all('table')


In [4]:
# loop over each table
for num, table in enumerate(tables, start=1):

1 <table cellpadding="0" cellspacing="0" class="tbls01" width="100%"><tr><th colspan="8" style="text-align:center;font-size:10pt;color:blue;">Daily Trends in FPI Investments on 07-Jun-2019</th></tr><tr><th align="center" width="100px">Reporting <br/>Date</th><th>Debt/Equity/Hybrid</th><th nowrap="">Investment Route</th><th>Gross Purchases(Rs. Crore)</th><th>Gross Sales (Rs. Crore)</th><th>Net Investment (Rs. Crore)</th><th>Net Investment US($) million</th><th>Conversion <br/>(1 USD TO INR)*</th></tr><tr><td rowspan="10">07-Jun-2019</td><td rowspan="3">Equity</td><td>Stock Exchange</td><td>6958.21</td><td>8517.98</td><td>(1559.77)</td><td>(225.01)</td><td rowspan="10"> Rs.69.3205</td></tr><tr><td>Primary market &amp; others</td><td>0.54</td><td>6.65</td><td>(6.11)</td><td>(0.88)</td></tr><tr><td>Sub-total</td><td>6958.75</td><td>8524.63</td><td>(1565.88)</td><td>(225.89)</td></tr><tr><td rowspan="3">Debt</td><td>Stock Exchange</td><td>3207.09</td><td>1687.67</td><td>1519.42</td><td>219.

In [0]:

# loop over each table
for num, table in enumerate(tables, start=1):
    # create filename
    filename = 'table-%d.csv' % num
    # open file for writing
    with open(filename, 'w') as f:
        # store rows here
        data = []
        # create csv writer object
        csv_writer = writer(f)
        # go through each row
        rows = table.find_all('tr')
        for row in rows:
            # write headers if any
            headers = row.find_all('th')
            if headers:
                csv_writer.writerow([header.text.strip() for header in headers])
            # write column items
            columns = row.find_all('td')
            csv_writer.writerow([column.text.strip() for column in columns])

In [6]:
import pandas as pd
df = pd.read_csv('table-1.csv')
df.head()

,,,,,,,Daily Trends in FPI Investments on 07-Jun-2019
Reporting Date,Debt/Equity/Hybrid,Investment Route,Gross Purchases(Rs. Crore),Gross Sales (Rs. Crore),Net Investment (Rs. Crore),Net Investment US($) million,Conversion (1 USD TO INR)*
07-Jun-2019,Equity,Stock Exchange,6958.21,8517.98,(1559.77),(225.01),Rs.69.3205
Primary market & others,0.54,6.65,(6.11),(0.88),NaN,NaN,NaN
Sub-total,6958.75,8524.63,(1565.88),(225.89),NaN,NaN,NaN
Debt,Stock Exchange,3207.09,1687.67,1519.42,219.19,NaN,NaN


In [0]:
!pip install quandl

In [0]:

 
import pandas as pd
from pandas import DataFrame
import argparse
import quandl
import pandas_datareader as web
from time import sleep
import datetime as dt
import sys
 
def get_sp500_data(start=dt.datetime.strptime("1997-01-01", "%Y-%m-%d"),
                   end=dt.datetime.now(), use_quandl=True, adjust=True, inner=True,
                   sleeptime=2, verbose=True):
    """Fetches S&P 500 data
     
    args:
        start: datetime; The earliest possible date
        end: datetime; The last possible date
        use_quandl: bool; Whether to fetch data from Quandl (reverts to Google if False)
        adjust: bool; Whether to use adjusted close (only works with Quandl)
        inner: bool; Whether to use an inner join or outer join when combining series (inner has no missing data)
        sleeptime: int; How long to sleep between fetches
        verbose: bool; Whether to print a log while fetching data
     
    return:
        DataFrame: Contains stock price data
    """
     
    join = "outer"
    if inner:
        join = "inner"
     
    symbols_table = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
                                 header=0)[0]
    symbols = list(symbols_table.loc[:, "Ticker symbol"])
 
    sp500 = None
    for s in symbols:
        sleep(sleeptime)
        if verbose:
            print("Processing: " + s + "...", end='')
        try:
            if use_quandl:
                s_data = quandl.get("WIKI/" + s, start_date=start, end_date=end)
                if adjust:
                    s_data = s_data.loc[:, "Adj. Close"]
                else:
                    s_data = s_data.loc[:, "Close"]
            else:
                s_data = web.DataReader(s, "google", start, end).loc[:, "Close"]
            s_data.name = s
            s_data.dropna()
            if s_data.shape[0] > 1:
                if sp500 is None:
                    sp500 = DataFrame(s_data)
                else:
                    sp500 = sp500.join(s_data, how=join)
                if verbose:
                    print(" Got it! From", s_data.index[0], "to", s_data.index[-1])
            else:
                if verbose:
                    print(" Sorry, but not this one!")
        except Exception:
            if verbose:
                print(" Sorry, but not this one!")
 
    badsymbols = list(set(s) - set(sp500.columns))
    if verbose and len(badsymbols) > 0:
        print("There were", len(badsymbols), "symbols for which data could not be obtained.")
        print("They are:", ", ".join(badsymbols))
     
    return sp500
 
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="Fetches S&P 500 data")
    parser.add_argument("-v", "--verbose", action="store_true", default=True, dest="verbose",
                        help="Print extra output [default]")
    parser.add_argument("--quietly", action="store_false",
                        dest="verbose", help="Don't print extra output")
    parser.add_argument("-f", "--file", type=str, dest="csv_name",
                        default="sp-500.csv",
                        help="CSV file to save data to [default: sp-500.csv]")
    parser.add_argument("-s", "--sleep", type=int, dest="sleeptime",
                        default=2,
                        help="Time (seconds) between fetching symbols [default: 2] (don't flood websites with requests!)")
    parser.add_argument("--inner", action="store_true", default=False, dest="inner",
                        help="Inner join; only dates where all symbols have data will be included")
    parser.add_argument("--start", type=str, dest="start",
                        default="1997-01-01",
                        help="Earliest date (YYYY-MM-DD) to include [default: 1997-01-01]")
    parser.add_argument("--end", type=str, dest="end",
                        default="today",
                        help='Last date (YYYY-MM-DD or "today") to include [default: "today"]')
    # parser.add_argument("-k", "--key", type="character", dest="api_key",
    #                     default=NULL,
    #                     help="Quandl API key, needed if getting Quandl data")
    parser.add_argument("-q", "--quandl", action="store_true", default=False,
                        dest="use_quandl", help="Get data from Quandl")
    parser.add_argument("-a", "--adjust", action="store_true", default=False,
                        dest="adjust", help="Adjust prices (Quandl only)")
    parser.add_argument("--about", action="store_true", default=False,
                        dest="about",
                        help="Print information about the script and its usage, then quit")
 
    args = parser.parse_args()
 
    if args.about:
        print(sys.argv[0], "\n(c) 2017 Curtis Miller\n",
          "Licensed under GNU GPL v. 3.0 available at ",
          "https://www.gnu.org/licenses/gpl-3.0.en.html \n",
          "E-mail: cgmil@msn.com\n\n",
          "This script fetches closing price data for ticker symbols included",
          "in the S&P 500 stock index. A list of symbols included in the index",
          "is fetched from this webpage:",
          "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies  The list",
          "is parsed and the symbols included in the list are fetched from",
          "either Google Finance (the default) or Quandl.",
          "If Quandl is the data source, adjusted data can be",
          "fetched instead. The resulting data set is then saved to a CSV",
          "file in the current working directory.\n\n",
          "This package requires the following Python packages be installed in order",
          "to work (all of which are available through pip):\n\n",
          "* pandas\n",
          "* pandas-datareader\n",
          "* quandl\n\n",
          "This script was written by Curtis Miller and was made available on ",
          "his website: https://ntguardian.wordpress.com\n\n",
          "You can read more about this script in the following article: ",
          "https://ntguardian.wordpress.com/blog\n\n")
        quit()
 
    if args.end == "today":
        args.end = dt.datetime.now()
    else:
        args.end = dt.datetime.strptime(args.end, "%Y-%m-%d")
    args.start = dt.datetime.strptime(args.start, "%Y-%m-%d")
    sp500 = get_sp500_data(start=args.start,
                   end=args.end, use_quandl=args.use_quandl, adjust=args.adjust, inner=args.inner,
                   sleeptime=args.sleeptime, verbose=args.verbose)
    sp500.to_csv(args.csv_name)

In [27]:
import pandas_datareader.data as pdweb
from pandas_datareader import data as pdr
import fix_yahoo_finance # must pip install first 
data = pdr.get_data_yahoo('SPY','2017-05-20','2017-05-30')
data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-05-22,239.710007,238.820007,238.899994,239.520004,61010600,230.560089
2017-05-23,240.240005,239.509995,239.949997,240.050003,48341700,231.070282
2017-05-24,240.729996,239.929993,240.320007,240.610001,46927700,231.609329
2017-05-25,242.080002,240.960007,241.199997,241.759995,64071700,232.716309
2017-05-26,241.899994,241.449997,241.539993,241.710007,46629900,232.668213
2017-05-30,241.789993,241.160004,241.339996,241.500000,35201900,232.466034


In [15]:
import pandas as pd
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
print(len(data) , type(data))
table = data[0]
print(table.shape)
table.head()

2 <class 'list'>
(505, 9)


,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [20]:
sliced_table = table[1:]
header = table.iloc[0]     #take one row and all columns
header

Symbol                                        MMM
Security                               3M Company
SEC filings                               reports
GICS Sector                           Industrials
GICS Sub Industry        Industrial Conglomerates
Headquarters Location         St. Paul, Minnesota
Date first added                              NaN
CIK                                         66740
Founded                                      1902
Name: 0, dtype: object

In [21]:
corrected_table = sliced_table.rename(columns=header)
corrected_table.head()

,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",nan,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
5,ATVI,Activision Blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",2015-08-31,718877,2008


In [0]:
#!pip install yahoo_fin
!pip install requests_html

In [6]:
# import stock_info module from yahoo_fin
from yahoo_fin import stock_info as si
# get live price of Apple
print(si.get_live_price("aapl"))
 
# or Amazon
print(si.get_live_price("amzn"))
# get quote table back as a data frame
print(si.get_quote_table("aapl", dict_result = False))
 
# or get it back as a dictionary (default)
# si.get_quote_table("aapl")
# get most active stocks on the day
# print(si.get_day_most_active())
 
# # get biggest gainers
# print(si.get_day_gainers())
 
# # get worst performers
# print(si.get_day_losers())

190.14999389648438
1804.030029296875
                   attribute                       value
0              1y Target Est                      216.17
1              52 Week Range             142.00 - 233.47
2                        Ask               190.15 x 1000
3                Avg. Volume                  3.0289e+07
4          Beta (3Y Monthly)                        1.03
5                        Bid                190.25 x 800
6                Day's Range             185.78 - 191.92
7                  EPS (TTM)                       11.89
8              Earnings Date  Jul 29, 2019 - Aug 2, 2019
9           Ex-Dividend Date                  2019-05-10
10  Forward Dividend & Yield                3.08 (1.76%)
11                Market Cap                    874.895B
12                      Open                      186.51
13            PE Ratio (TTM)                          16
14            Previous Close                      185.22
15               Quote Price                      1

In [0]:
#Get all the prices(open to close) for each company 
import bs4 as bs
import datetime as dt
import os 
import pandas as pd
import pandas_datareader.data as web
import pickle
import requests

# This is a script that will get the tickers from the wikipedia list, them and save it as a pickle file"
def save_sp500_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers= []
    for row in table.findAll('tr')[1:]:
        ticker=row.findAll('td')[0].text.split('\n')[0]
        #print(ticker)
        mapping = str.maketrans(".","-")
        ticker = ticker.translate(mapping)
        if ticker.find("_"):
            ticker = ticker.replace("_","-")
        tickers.append(ticker)
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)

    return tickers

print(save_sp500_tickers())
def get_data_from_yahoo(reload_sp500 = False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else: 
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2000,1,1)
    end = dt.datetime(2017,12,31)

    for ticker in tickers:
        #print(ticker)
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker.replace(".","-"), 'yahoo', start, end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))

get_data_from_yahoo()


In [35]:
import pandas as pd
import pandas_datareader.data as web

symbols_list = ['ORCL', 'TSLA', 'IBM','YELP', 'MSFT','FB','GOOGL']
d = {}
for ticker in symbols_list:
    d[ticker] = web.DataReader(ticker, "yahoo", '2014-12-01')
pan = pd.Panel(d)
df1 = pan.minor_xs('Adj Close')
df1.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)


,ORCL,TSLA,IBM,YELP,MSFT,FB,GOOGL
Date,,,,,,,
2014-12-01,39.252628,231.639999,133.261368,55.130001,43.943386,75.099998,539.650024
2014-12-02,39.345913,231.429993,134.193604,55.169998,43.798779,75.459999,538.590027
2014-12-03,39.233971,229.300003,135.719757,54.770000,43.455330,74.879997,536.969971
2014-12-04,39.075394,228.279999,135.332016,55.139999,44.142227,75.239998,542.580017
2014-12-05,39.112701,223.710007,134.688553,53.840000,43.762627,76.360001,528.080017


In [36]:
df_percent_chg = df1.pct_change()
df_percent_chg.head()

,ORCL,TSLA,IBM,YELP,MSFT,FB,GOOGL
Date,,,,,,,
2014-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-02,0.002377,-0.000907,0.006996,0.000726,-0.003291,0.004794,-0.001964
2014-12-03,-0.002845,-0.009204,0.011373,-0.007250,-0.007842,-0.007686,-0.003008
2014-12-04,-0.004042,-0.004448,-0.002857,0.006756,0.015807,0.004808,0.010448
2014-12-05,0.000955,-0.020019,-0.004755,-0.023576,-0.008599,0.014886,-0.026724


In [27]:
import pandas_datareader.data as web
import datetime    

start = datetime.datetime(2013, 1, 1)
end = datetime.datetime(2016, 1, 27)
df = web.DataReader("GOOGL", 'yahoo', start, end)


#Change the datatype of the Data(into string)
dates =[]
for x in range(len(df)):
    newdate = str(df.index[x])
    newdate = newdate[0:10]
    dates.append(newdate)

df['dates'] = dates
df.head()

,High,Low,Open,Close,Volume,Adj Close,dates
Date,,,,,,,
2013-01-02,363.863861,358.633636,360.070068,361.987000,5077500.0,361.987000,2013-01-02
2013-01-03,366.331329,360.720734,362.827820,362.197205,4631700.0,362.197205,2013-01-03
2013-01-04,371.106110,364.204193,365.035034,369.354340,5521400.0,369.354340,2013-01-04
2013-01-07,370.060059,365.655670,368.093079,367.742737,3308000.0,367.742737,2013-01-07
2013-01-08,368.518524,362.577576,368.138153,367.017029,3348800.0,367.017029,2013-01-08


In [20]:
from datetime import datetime
stocks = ['ORCL', 'TSLA', 'IBM','YELP', 'MSFT']
ls_key = 'Adj Close'
start = datetime(2014,1,1)
end = datetime(2014,3,28)    
print(start , end)
f = web.DataReader(stocks, 'yahoo',start,end)
#cleanData = f.ix[ls_key]
dataFrame = pd.DataFrame(f)

dataFrame.head()

2014-01-01 00:00:00 2014-03-28 00:00:00


Attributes        High                        ...  Adj Close                       
Symbols            IBM       MSFT       ORCL  ...       ORCL        TSLA       YELP
Date                                          ...                                  
2014-01-02  187.399994  37.400002  38.029999  ...  34.871212  150.100006  67.919998
2014-01-03  187.350006  37.220001  37.860001  ...  34.778778  149.559998  67.660004
2014-01-06  187.360001  36.889999  37.799999  ...  34.640110  147.000000  71.720001
2014-01-07  190.350006  36.490002  37.930000  ...  34.991413  149.360001  72.660004
2014-01-08  189.419998  36.139999  37.910000  ...  34.871231  151.279999  78.419998

[5 rows x 30 columns]

In [21]:
dataFrame.columns

MultiIndex(levels=[['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], ['IBM', 'MSFT', 'ORCL', 'TSLA', 'YELP']],
           codes=[[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5], [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4]],
           names=['Attributes', 'Symbols'])

In [37]:
from pandas_datareader import data as pdr
import fix_yahoo_finance

data = pdr.get_data_yahoo('BHP.AX', start='2017-04-23', end='2017-05-24')
data.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-04-24,24.160000,23.730000,24.139999,23.940001,4909695,21.847147
2017-04-26,24.240000,23.930000,24.100000,24.080000,8407298,21.974909
2017-04-27,24.200001,23.780001,24.200001,23.870001,6516105,21.783270
2017-04-28,23.799999,23.320000,23.389999,23.719999,11182018,21.646379
2017-05-01,23.940001,23.625000,23.700001,23.879999,6003898,21.792393


In [0]:
!pip install scrapy